In [1]:
# import geospatial packages
import ee
import folium
import geehydro

# import others
from datetime import datetime as dt
from IPython.display import Image

In [2]:
# connect to Earth Engine
ee.Initialize()

In [3]:
# we'll focus on an area in the amazon
amazon_map = folium.Map(location=[-4, -52], zoom_start=10)

In [4]:
amazon_map

In [5]:
# load in the landsat 8 assets, we could use other catalogs too
landsat8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

In [6]:
# define an area of interest, there are a range of ways of doing this
AOI = ee.Geometry.Rectangle([-51.84448, -3.92180, -52.23999, -4.38201])

In [7]:
# filter the landsat imagery spatially to just images that cover our AOI
landsat8_AOI = landsat8.filterBounds(AOI)

In [8]:
# print the number of images that match our search criteria
print('Total number:', landsat8_AOI.size().getInfo())

Total number: 106


In [9]:
# filter the landsat imagery temporally to some recent summer images
landsat8_AOI_summmer2019 = landsat8_AOI.filterDate('2019-07-01','2019-12-01')

In [10]:
# print the number of images that match our new search criteria
print('Total number:', landsat8_AOI_summmer2019.size().getInfo())

Total number: 9


In [11]:
# we should already know the bands but let's check that they are there
landsat8_AOI_summmer2019.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B10',
 'B11',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

In [12]:
# we have 9 images matching our criteria, so let's sort by cloud cover
fewest_clouds = ee.Image(landsat8_AOI_summmer2019.sort('CLOUD_COVER').first())

In [13]:
# let's see how cloudy our least cloudy image is
print("Image has {}% clouds.".format(fewest_clouds.get('CLOUD_COVER').getInfo()))

Image has 0% clouds.


In [14]:
# let's see when this image was taken
date = ee.Date(fewest_clouds.get('system:time_start'))

In [15]:
# what does this return?
print(date)

ee.Date({
  "functionInvocationValue": {
    "functionName": "Date",
    "arguments": {
      "value": {
        "functionInvocationValue": {
          "functionName": "Element.get",
          "arguments": {
            "object": {
              "functionInvocationValue": {
                "functionName": "Collection.first",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.limit",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "Collection.filter",
                            "arguments": {
                              "collection": {
                                "functionInvocationValue": {
                                  "functionName": "Collection.filter",
                                  "arguments": {
                                  

In [16]:
# we need to do some further parsing to get the actual date
time = date.getInfo()['value']/1000.

In [17]:
# now what do we have?
print(time)

1565530582.221


In [18]:
# that still doesn't look quite right, but this should be in UTC
# we can use the datetime library to make this human-readable
print(dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S'))

2019-08-11 13:36:22


In [19]:
# let's put together a simple RGB image of this area
parameters = {'min': 0,
              'max': 1000,
              'dimensions': 1000,
              'bands': ['B4', 'B3', 'B2'],
              'region': AOI}

# we can now query ee and display it as a static image in IPython
Image(url = fewest_clouds.getThumbUrl(parameters))

In [20]:
# calculate the NDVI quickly, or any other normalized difference reallyndvi = least_cloudy.normalizedDifference(['B5', 'B4'])
fewest_clouds_ndvi = fewest_clouds.normalizedDifference(['B5', 'B4'])

In [21]:
# since interactive maps are more fun than static images, let's make a map
# set a color map, we can use anythin here really
palette = ['red', 'yellow', 'green']

# set our NDVI parameters
ndvi_parameters = {'min': 0,
                   'max': 1,
                   'dimensions': 512,
                   'palette': palette,
                   'region': AOI}

# add the new layer to the map we created above
amazon_map.addLayer(fewest_clouds_ndvi, ndvi_parameters)
amazon_map